In [ ]:
# Importing necessary Python libraries:
import spacy  # For natural language processing (NER).
from collections import defaultdict  # For storing nested dictionaries of policy mentions.
import pandas as pd
import plotly.graph_objects as go  # For creating radar plots.

# Loading the pre-trained spaCy model for Named Entity Recognition (NER).
nlp = spacy.load("en_core_web_sm")

# Defining the policy data documents for each country with their respective file paths.
policy_docs = {
    'Australia': [
        "/content/Australia’s network of nuclear cooperation agreements.txt",
        "/content/Australia2023EnergyPolicyReview_extracted_paragraphs.txt",
        "/content/AustraliaPublicPolicy.txt"
    ],
    'France': [
        "/content/Energy_Policy_France_2016_Review_extracted_paragraphs.txt"
    ],
    'Singapore': [
        "/content/singapore_nr_extracted_paragraphs.txt",
        "/content/SingaporePublicPolicy.txt"
    ]
}

# Initializing a nested dictionary to store the count of policy mentions by country and policy type.
policy_mentions = defaultdict(lambda: defaultdict(int))

# Defining a list of energy policies to search for within the documents.
policies = [
    "nuclear", "solar", "natural gas", "hydrogen", "oil",
    "wind", "biomass", "hydro", "hydrogen"
]

# Iterating over each country and its associated policy documents.
for country, files in policy_docs.items():
    combined_text = ""

    # Read and combines the contents of each policy document for the relevant country.
    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            combined_text += f.read()

    # Processing the combined text using the spaCy NLP model.
    doc = nlp(combined_text)

    # Searching for mentions of each policy term in the combined text.
    for policy in policies:
        count = combined_text.lower().count(policy.lower())  # Case-insensitive counting
        policy_mentions[country][policy] += count  # Storing the count in the dictionary.

# Preparing the data for the radar plot by organizing it into a list of dictionaries.
radar_data = []
for country, mentions in policy_mentions.items():
    data_row = {'Country': country}  # Initializing a row with the country name.
    for policy in policies:
        data_row[policy] = mentions.get(policy, 0)  # Storing the count of each policy mention.
    radar_data.append(data_row)  # Adding the row to the radar data.

# Converting the radar data into a DataFrame for easier plotting.
df_radar = pd.DataFrame(radar_data)

# Creating the radar plot using Plotly.
fig = go.Figure()

# Adding a trace to the radar plot for each country.
for i in range(len(df_radar)):
    fig.add_trace(go.Scatterpolar(
        r=df_radar.iloc[i, 1:],  # Selecting all columns except the 'Country' column.
        theta=df_radar.columns[1:],  # Using the policy names as the angular coordinates (theta).
        fill='toself',  # Filling the area inside the trace.
        name=df_radar['Country'][i]  # Using the country name as the trace label.
    ))

# Customizing the layout of the radar plot.
fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,  # Showing the radial axis.
            range=[0, df_radar.drop(columns='Country').max().max()]  # Setting the range based on the max value.
        )
    ),
    showlegend=True,  # Displaying the legend.
    title='Energy Policy Influence Across Countries (Radar Plot)'  # Setting the plot title.
)

# Displaying the radar plot.
fig.show()

